In [1]:
import json



Import JSON from dataset file that is output by the C# application.

In [2]:
with open('..\Output\MethodCallsWithoutSource.json') as json_file:
    data = json.load(json_file)
    for test_case_index in range (0, 5):
        print('Class Name:', data[test_case_index]['ClassName'])
        print('Is Flawed:', data[test_case_index]['IsFlawed'])
        print(type(data[test_case_index]['IsFlawed']))
        print('Features:', data[test_case_index]['Features'])
        print('-----------------------------------------------')

Class Name: CWE078
Is Flawed: True
<class 'bool'>
Features: System.Console.WriteLine(string)
System.Console.ReadLine()
System.Console.WriteLine(string)
System.Diagnostics.Process.Start(string)
System.Console.WriteLine(string)

-----------------------------------------------
Class Name: CWE078
Is Flawed: True
<class 'bool'>
Features: System.Console.WriteLine(string)
System.Console.ReadLine()
System.Console.WriteLine(string)
OS_Command_Injection_scope_78.excuteCommand(string)
System.Diagnostics.Process.Start(string)
System.Console.WriteLine(string)

-----------------------------------------------
Class Name: CWE078
Is Flawed: True
<class 'bool'>
Features: System.Console.WriteLine(string)
System.Console.ReadLine()
System.Console.WriteLine(string)
System.Diagnostics.Process.Start(string)
System.Console.WriteLine(string)

-----------------------------------------------
Class Name: CWE089
Is Flawed: True
<class 'bool'>
Features: System.Data.SqlClient.SqlConnection.CreateCommand()
System.Data

In [3]:
print(len(data))
categories = ['No Flaw', 'CWE022', 'CWE078', 'CWE089', 'CWE090', 'CWE091']
labels = []
examples = []
examples_skipped = 0

for test_case_index in range(0, len(data)):
    if data[test_case_index]['IsFlawed']:
        try:
            category_index = categories.index(data[test_case_index]['ClassName'])
            labels.append(category_index)
            examples.append(data[test_case_index]['Features']) 
        except ValueError:
            examples_skipped = examples_skipped + 1
    else:
        labels.append(0)
        examples.append(data[test_case_index]['Features'])
        
print('Label Count:', len(labels))
print('Example Count:', len(examples))
print('Examples Skipped:', examples_skipped)

30980
Label Count: 30970
Example Count: 30970
Examples Skipped: 10


In [4]:
for test_case_index in range (0, 5):
    print('Label:', labels[test_case_index])
    print('Category:', categories[labels[test_case_index]])
    print('Example:', examples[test_case_index])

Label: 2
Category: CWE078
Example: System.Console.WriteLine(string)
System.Console.ReadLine()
System.Console.WriteLine(string)
System.Diagnostics.Process.Start(string)
System.Console.WriteLine(string)

Label: 2
Category: CWE078
Example: System.Console.WriteLine(string)
System.Console.ReadLine()
System.Console.WriteLine(string)
OS_Command_Injection_scope_78.excuteCommand(string)
System.Diagnostics.Process.Start(string)
System.Console.WriteLine(string)

Label: 2
Category: CWE078
Example: System.Console.WriteLine(string)
System.Console.ReadLine()
System.Console.WriteLine(string)
System.Diagnostics.Process.Start(string)
System.Console.WriteLine(string)

Label: 3
Category: CWE089
Example: System.Data.SqlClient.SqlConnection.CreateCommand()
System.Data.SqlClient.SqlParameterCollection.Add(System.Data.SqlClient.SqlParameter)
System.Data.SqlClient.SqlParameterCollection.Add(System.Data.SqlClient.SqlParameter)
System.Data.SqlClient.SqlCommand.ExecuteNonQuery()
System.Console.WriteLine(string)
S

In [5]:
# This tells matplotlib not to try opening a new window for each plot.

%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt
from scipy import misc

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import *
from sklearn.feature_extraction import *
###########
num_test = int(len(examples) / 2)
train_data, train_labels = examples[num_test:], labels[num_test:]
test_data, test_labels = examples[:num_test], labels[:num_test]

vectorizer = CountVectorizer(vocabulary={'No Flaw':0, 'CWE022':1, 'CWE078':2, 'CWE089':3, 'CWE090':4, 'CWE091':5})



vtrain = vectorizer.transform(examples)
print('Non-zero features per example:', 1.0 * vtrain.nnz / vtrain.shape[0])

Non-zero features per example: 0.0


In [6]:
# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB



# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

#Perform KNN on training data
cv = CountVectorizer()
Q3 = cv.fit_transform(train_data)
#FN=cv.get_feature_names(Q3)
model = KNeighborsClassifier(n_neighbors=5).fit(Q3, train_labels) #used guess and check for best K value

Q3test_data = cv.transform(test_data)
predicted_labels = model.predict(Q3test_data)
print("KNN F1: ", f1_score(test_labels, predicted_labels, average='weighted'))
print("KNN Accuracy: ", accuracy_score(predicted_labels, test_labels))

Q3mnb = cv.transform(train_data)
mnbModel = MultinomialNB(alpha=.001).fit(Q3mnb, train_labels) #used guess and check for alpha value
Q4test_data = cv.transform(test_data)
predicted_labels_mnb = mnbModel.predict(Q4test_data)
print("MNB F1: ", f1_score(test_labels, predicted_labels_mnb, average='weighted'))
print("MNB Accuracy: ", accuracy_score(predicted_labels_mnb, test_labels))

#logistic regression model 
#Q5train_data = cv.transform(train_data)
XLR = cv.transform(train_data)
LR = LogisticRegression().fit(XLR,train_labels)
Q5test_data = cv.transform(test_data)
predicted_labels_lr = LR.predict(Q5test_data)
print("LR F1: ", f1_score(test_labels, predicted_labels_lr, average='weighted'))
print("LR Accuracy: ", accuracy_score(predicted_labels_lr, test_labels))

matrix = confusion_matrix(test_labels, predicted_labels_lr)
print(matrix)
max_errors = 0
most_confused_actual = -1
most_confused_predicted = -1
# Find the most confused digit pair
for actual in range(0, len(categories)):
    for predicted in range(0, len(categories)):
        if (actual != predicted and matrix[actual, predicted] > max_errors):
            max_errors = matrix[actual, predicted]
            most_confused_actual = actual
            most_confused_predicted = predicted
print ("Most confused pair is actual =", categories[most_confused_actual], ", predicted =", categories[most_confused_predicted])
print ("This error occurred", max_errors, "times")
# Print out examples of the confused digits
print(type(most_confused_actual))
print(type(test_labels[0]))
error_pair_count = 0
for index in range(0, len(test_labels)):
    if (test_labels[index] == most_confused_actual and predicted_labels_lr[index] == most_confused_predicted and error_pair_count < 10):
        print(test_data[index])
        print("-----------------------------")
        error_pair_count = error_pair_count + 1

C:\Users\chris\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\chris\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


KNN F1:  0.5176469278201159
KNN Accuracy:  0.48149822408782694


C:\Users\chris\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\chris\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


MNB F1:  0.5130213637492773
MNB Accuracy:  0.5721020342266709


C:\Users\chris\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\chris\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LR F1:  0.527965906262004
LR Accuracy:  0.5963835970293833
[[ 803  270    0 4290    0    0]
 [   0    0    0    0    0    0]
 [ 225  621    0   79  107  177]
 [ 481    0    0 8432    0    0]
 [   0    0    0    0    0    0]
 [   0    0    0    0    0    0]]
Most confused pair is actual = No Flaw , predicted = CWE089
This error occurred 4290 times
<class 'int'>
<class 'int'>
System.Diagnostics.Process.Start()
System.IO.StreamReader.ReadToEnd()
System.Diagnostics.Process.WaitForExit()
System.Diagnostics.Process.Close()
System.Security.Cryptography.HashAlgorithm.ComputeHash(byte[])
System.Text.Encoding.GetBytes(string)
System.Text.StringBuilder.Append(string)
byte.ToString(string)
System.Text.StringBuilder.ToString()

-----------------------------
System.Console.ReadLine()
System.Security.Cryptography.HashAlgorithm.ComputeHash(byte[])
System.Text.Encoding.GetBytes(string)
System.Text.StringBuilder.Append(string)
byte.ToString(string)
System.Text.StringBuilder.ToString()

-----------------